Project: Tweets analysis of the main Italian populist politicians - Giuseppe Grillo, Giorgia Meloni and Matteo Salvini.

Student: **Alessandro Monolo** | 1790210

Lecturer: Erik HEKMAN

Fundamentals of Data Science - Master Data-Driven Design, Hogeschool Utrecht.

August 2021 - Block E

In [ ]:
# Here I import the libraries I need

In [1]:
import tweepy
import requests
import json
import webbrowser
import time
import pandas as pd

In [ ]:
# Here I save my personal twitter API secret keys

In [2]:
consumer_key = "zEOrE8aIMeJDVYQr09miZYS84"
consumer_secret = "BsRCsGxPue6VgN2PiPcqHAbvOLtdYRynwWYOMXjcodju9Yie5i"
access_token = "1400895858938462211-PQaaX3vGL71KOhDr0lCLNI72c7lMkJ"
access_token_secret = "5xVWuUIAEn4U4DfeeMNeSWPhHPLLNWG2F59tjov4loUKK"

In [3]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

In [4]:
api = tweepy.API(auth, wait_on_rate_limit = True)

In [ ]:
# Declare the twitter user profile where to get the last tweets using tweepy as scraping library

In [5]:
userID = "beppe_grillo"

In [6]:
tweets = api.user_timeline(screen_name=userID, 
                           # 200 is the maximum allowed count
                           count=200,
                           include_rts = False,
                           # Necessary to keep full_text 
                           # otherwise only the first 140 words are extracted
                           tweet_mode = 'extended'
                           )

In [ ]:
# Test if everything is working smoothly

In [7]:
for info in tweets[:3]:
     print("ID: {}".format(info.id))
     print(info.created_at)
     print(info.full_text)
     print("\n")

ID: 1423947184282681344
2021-08-07 10:00:39
Il suolo è una risorsa preziosa che finora non abbiamo protetto. Ecco il giro d'Italia della #CompagniaDelSuolo per conoscere lo stato di salute dei suoli nei nostri campi. https://t.co/n6tlJVg6i6


ID: 1423230797121740803
2021-08-05 10:33:59
Preferiamo perdere un dito o un braccio? Ho qualcosa di importante da dire a tutti voi. https://t.co/JijbzMPihf


ID: 1422504336291139594
2021-08-03 10:27:17
L’agricoltura sulla Terra ha da sempre preso le forme e le modalità più strane, presentandosi in molte forme diverse. Ora, un gruppo di ricerca ha creato il primo orto subacqueo al mondo. Pensate, dove si trova? In Italia! https://t.co/P7hAyuiaSE




In [8]:
all_tweets = []
all_tweets.extend(tweets)
oldest_id = tweets[-1].id

In [ ]:
# Get the last tweets of userID with tweet_mode set as extended in order to get the full
# text of each tweets and therefor not a cut one which has only 140 characters available per each tweet:

In [9]:
while True:
    tweets = api.user_timeline(screen_name=userID, 
                           # 200 is the maximum allowed count
                           count=200,
                           include_rts = False,
                           max_id = oldest_id - 1,
                           # Necessary to keep full_text 
                           # otherwise only the first 140 words are extracted
                           tweet_mode = 'extended'
                           )
    if len(tweets) == 0:
        break
    oldest_id = tweets[-1].id
    all_tweets.extend(tweets)
    print('N of tweets downloaded till now {}'.format(len(all_tweets)))

N of tweets downloaded till now 399
N of tweets downloaded till now 599
N of tweets downloaded till now 799
N of tweets downloaded till now 997
N of tweets downloaded till now 1197
N of tweets downloaded till now 1397
N of tweets downloaded till now 1589
N of tweets downloaded till now 1789
N of tweets downloaded till now 1989
N of tweets downloaded till now 2188
N of tweets downloaded till now 2387
N of tweets downloaded till now 2587
N of tweets downloaded till now 2787
N of tweets downloaded till now 2987
N of tweets downloaded till now 3186
N of tweets downloaded till now 3236


In [ ]:
# Store into a csv file the info of each tweets I need, such as text, datetime, number of retweets and favorite count:

In [10]:
from pandas import DataFrame
outtweets = [[tweet.id_str, 
              tweet.created_at, 
              tweet.favorite_count, 
              tweet.retweet_count, 
              tweet.full_text.encode("utf-8").decode("utf-8")] 
             for idx,tweet in enumerate(all_tweets)]
df = DataFrame(outtweets,columns=["id","created_at","favorite_count","retweet_count", "text"])
df.to_csv('%s_tweets.csv' % userID,index=False)
df.head(211)

,id,created_at,favorite_count,retweet_count,text
0,1423947184282681344,2021-08-07 10:00:39,73,25,Il suolo è una risorsa preziosa che finora non...
1,1423230797121740803,2021-08-05 10:33:59,205,88,Preferiamo perdere un dito o un braccio? Ho qu...
2,1422504336291139594,2021-08-03 10:27:17,190,57,L’agricoltura sulla Terra ha da sempre preso l...
3,1422107888735170561,2021-08-02 08:11:56,198,69,Immaginate un mondo digitale in cui potete con...
4,1421399023357992967,2021-07-31 09:15:10,264,84,La #Pediatria di #Padova vince in umanità: i #...
...,...,...,...,...,...
206,1284035280962564096,2020-07-17 08:00:41,493,161,Onde evitare strani appostamenti ed inseguimen...
207,1283648583863017475,2020-07-16 06:24:06,409,150,"Il modo in cui produciamo, consumiamo e smalti..."
208,1282281011729006598,2020-07-12 11:49:51,1443,508,"Virgì, Roma nun te merita https://t.co/3vx37oSRgw"
209,1281169531780714496,2020-07-09 10:13:14,213,90,#Giappone: più felicità e meno divorzi con un ...
